Tiếp tục xử lý dữ liệu

# Khai báo thư viện

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import datetime

# Đọc dữ liệu

In [2]:
df = pd.read_csv('data4884.csv')
df.head(2)

,Unnamed: 0,Name,Brand,Announced,Status,Size,Resolution,OS,Chipset,Card slot,...,RAM,Num of MC,Pic MC,Video MC,Num of SC,Pic SC,Loudspeaker,3.5mm jack,Battery,Price
0,0,1T 7,alcatel,"2018, February","Available. Released 2018, June","7.0 inches, 137.9 cm2 (~65.6% screen-to-body r...","600 x 1024 pixels, 16:9 ratio (~170 ppi density)",Android 8.1 (Oreo),Mediatek MT8321A/D (28 nm),microSDXC (dedicated slot),...,1.0,1.0,2,720p@30fps,1.0,0.3,Yes,Yes,2580.0,About 70 EUR
1,1,Idol 5s,alcatel,"2017, September","Available. Released 2017, September","5.2 inches, 74.5 cm2 (~69.7% screen-to-body ra...","1080 x 1920 pixels, 16:9 ratio (~424 ppi density)",Android 7.1 (Nougat),Mediatek MT6757 Helio P20 (16 nm),microSDXC (dedicated slot),...,3.0,1.0,12,1080p@30fps,1.0,8,"Yes, with stereo speakers",Yes,2850.0,About 140 EUR


In [3]:
df.shape

(4884, 21)

Số lượng bản ghi còn 4884, cột mất đi ``Video SC`` tuy nhiên vẫn để lại cột chứa index là ``Unnamed: 0`` để lấy chỉ số và cào dữ liệu lại nếu cần 

# Các vấn đề về dữ liệu
Đã xử lý xong trùng lặp và giá trị bị thiếu

Thay tên cột ``Unnamed: 0`` thành ``ID``

In [4]:
df.rename(columns = {'Unnamed: 0':'ID'},inplace=True)

## Kiểu dữ liệu

In [5]:
df.dtypes

ID              int64
Name           object
Brand          object
Announced      object
Status         object
Size           object
Resolution     object
OS             object
Chipset        object
Card slot      object
ROM            object
RAM            object
Num of MC      object
Pic MC         object
Video MC       object
Num of SC      object
Pic SC         object
Loudspeaker    object
3.5mm jack     object
Battery        object
Price          object
dtype: object

- ``ROM, RAM, Num of MC/SC, Pic MC/SC, Video MC, Battery, Price`` cần xử lý để chuyển về kiểu số nguyên hoặc số thực
- ``Status, Card slot, Loudspeaker, 3.5mm jack`` cũng có thể chuyển về dạng 0,1 nếu cần

### Announced và Status
2 cột này xuất hiện thêm thời gian phát hành (Released)

In [6]:
df.Announced.value_counts()

2018, February                                          38
2019, September                                         35
2018, May                                               35
2014, September. Released 2014, September               35
2018, October                                           33
                                                        ..
2019, February 07                                        1
2012, May. Released 2012, July                           1
2011, September. Released Exp. release 2011, October     1
2019, May 15                                             1
2011, December. Released 2012, February                  1
Name: Announced, Length: 1407, dtype: int64

- Cột ``Announced`` khá nhiều thông tin dư thừa (năm, quý, ngày tháng, thời gian phát hành (Released) , ... ) 
- Nên xử lý bằng cách chỉnh về dạng format <năm>, <quý>

In [7]:
df.Status.value_counts()

Discontinued                           2143
Available. Released 2018, April          41
Available. Released 2019, September      41
Available. Released 2018, May            39
Available. Released 2018, June           37
                                       ... 
Available. Released 2020, June 04         1
Available. Released 2020, March 15        1
Available. Released 2020, May 01          1
Available. Released 2020, March 23        1
Available. Released 2016, April           1
Name: Status, Length: 927, dtype: int64

- Cột ``Status`` lưu thông tin về tình trạng sản phẩm. Tuy nhiên giá trị *Available* có thêm thời gian phát hành

Cần loại bỏ thông tin thời gian phát hành khỏi 2 cột

### Size

In [8]:
df.Size.value_counts()

5.0 inches, 68.9 cm2                                    19
6.51 inches, 102.3 cm2 (~81.6% screen-to-body ratio)    18
5.0 inches, 68.9 cm2 (~66.7% screen-to-body ratio)      18
6.5 inches, 102.0 cm2 (~81.6% screen-to-body ratio)     17
5.0 inches, 68.9 cm2 (~66.0% screen-to-body ratio)      17
                                                        ..
4.0 inches, 45.5 cm2 (~63.7% screen-to-body ratio)       1
5.3 inches, 77.4 cm2                                     1
5.5 inches, 83.4 cm2 (~80.5% screen-to-body ratio)       1
5.45 inches, 76.7 cm2 (~71.5% screen-to-body ratio)      1
7.0 inches, 137.9 cm2 (~56.0% screen-to-body ratio)      1
Name: Size, Length: 2493, dtype: int64

Có chứa 3 thông tin: Kích thước màn hình, Diện tích màn hình và Phần trăm diện tích màn hình so với diện tích thân máy. Tuy nhiên 2 thông tin đầu có thể mọi máy đều có. Vì vậy có thể tách làm 2 cột nếu không xảy ra tình trạng missing value hoặc chỉ lấy giá trị không bị missing value

In [9]:
inches = []
area = []
idx = 0
for x in df.Size:
    try:
        # Lấy thông tin số trong chuỗi
        i,s = x.split(' inches')
        a = re.findall(r"(\d*\.*\d) cm2", s)[0]
        inches.append(i)
        area.append(a)
    except:
        # In ra vị trí bị lỗi
        print(idx)
    idx+=1
len(inches), len(area)

2621


(4883, 4883)

- 4883 bản ghi đều có đủ kích thước và diện tích màn hình.
- Tiến hành kiểm tra dòng 2621.

In [10]:
df.loc[[2621]]

,ID,Name,Brand,Announced,Status,Size,Resolution,OS,Chipset,Card slot,...,RAM,Num of MC,Pic MC,Video MC,Num of SC,Pic SC,Loudspeaker,3.5mm jack,Battery,Price
2621,7000,Name,Brand,Announced,Status,Size,Resolution,OS,Chipset,Card slot,...,RAM,Num of MC,Pic MC,Video MC,Num of SC,Pic SC,Loudspeaker,3.5mm jack,Battery,Price


Xuất hiện 1 dòng có ID là 7000 có tất cả giá trị trùng với tên cột. Đây có thể là do quá trình đọc ghi file dữ liệu. Cần xoá bỏ bản ghi này

### Resolution
Cột này mang 3 thông tin là độ phân giải, tỉ lệ màn hình và mật độ điểm ảnh. Tỉ lệ màn hình có thể tính dựa vào độ phân giải.

In [11]:
df.Resolution.value_counts()

720 x 1280 pixels, 16:9 ratio (~294 ppi density)         321
1080 x 1920 pixels, 16:9 ratio (~401 ppi density)        176
720 x 1600 pixels, 20:9 ratio (~270 ppi density)         158
480 x 800 pixels, 5:3 ratio (~233 ppi density)           153
720 x 1280 pixels, 16:9 ratio (~267 ppi density)         136
                                                        ... 
1080 x 2340 pixels, 19.5:9 ratio (~413 ppi density)        1
1080 x 2400 pixels, 20:9 ratio (~415 ppi density)          1
1080 x 2400 pixels, 20:9 ratio (~429 ppi density)          1
1080 x 2400 pixels, 20:9 ratio (~416 ppi density)          1
540 x 960 pixels (TBC), 16:9 ratio (~245 ppi density)      1
Name: Resolution, Length: 623, dtype: int64

In [12]:
pixels = []
ppi = []
error = []
idx = 0
for x in df.Resolution:
    try:
        p,s = x.split(' pixels')
        i = re.findall(r"(\d*\.*\d) ppi density", s)[0]
        pixels.append(p)
        ppi.append(i)
    except:
        # In ra vị trí bị lỗi
        error.append(idx)
    idx+=1
len(pixels), len(ppi), len(error)

(4878, 4878, 6)

Có 6 giá trị bị lỗi 

In [13]:
error

[584, 864, 869, 888, 2412, 2621]

Dòng 2621 là dòng bị lỗi trước đó

In [14]:
i=0
for v in df.Resolution.loc[error]:
    print(error[i],v,sep=': ')
    i+=1

584: 1080 x 1920 pixels720 x 1280 pixels - K550 model, 16:9 ratio (~386 ppi density)
864: 540 x 960, 16:9 ratio (~220 ppi density)
869: 540 x 960, 16:9 ratio (~220 ppi density)
888: 320 x 480 pixels, 3:2 ratio
2412: 1080 x 1920 pixels (KNT-AL10, KNT-TL10) (~386 ppi pixel density) 1440 x 2560 pixels (KNT-AL20), 16:9 ratio (~515 ppi density)
2621: Resolution


- 584 và 2412 có 2 phiên bản. Tuy nhiên chỉ lấy phiên bản đầu tiên
- 864 và 869 không có từ khoá pixels 
- 888 không có ppi

In [15]:
df.loc[[888]]

,ID,Name,Brand,Announced,Status,Size,Resolution,OS,Chipset,Card slot,...,RAM,Num of MC,Pic MC,Video MC,Num of SC,Pic SC,Loudspeaker,3.5mm jack,Battery,Price
888,2139,A80,Micromax,"2012, August. Released 2012, August",Discontinued,"3.75 inches, 41.9 cm2 (~50.5% screen-to-body r...","320 x 480 pixels, 3:2 ratio",Android 2.3.4 (Gingerbread),Mediatek MT6573 (65 nm),microSDHC (dedicated slot),...,0.0,1.0,5,Yes,1.0,0.3,Yes,Yes,2500.0,About 120 EUR


Dòng 888 có ppi là 156

### Card slot

In [16]:
df['Card slot'].value_counts()

microSDXC (dedicated slot)                                                                               1541
No                                                                                                        936
microSDXC (uses shared SIM slot)                                                                          774
microSDHC (dedicated slot)                                                                                754
microSDXC                                                                                                 396
microSDHC                                                                                                 202
NM (Nano Memory), up to 256GB (uses shared SIM slot)                                                       62
microSDXC (uses shared SIM slot) - dual SIM model only                                                     39
microSDHC (dedicated slot), 8 GB included                                                                  27
microSDHC 

- Có 21 giá trị Unspecified 
- Có 1 dòng Yes nhưng không rõ là loại thẻ nhớ nào
- Gồm có các loại thẻ sau: SD,miniSD,microSD,microSDHC,microSDXC,microSDUC, NM(của hãng Huawei)
- Ngoài ra, RS-DV-MMC là loại thẻ nhớ cũ (MultiMediaCard và các kiểu cùng loại)

In [17]:
df[df['Card slot'] =='Unspecified']

,ID,Name,Brand,Announced,Status,Size,Resolution,OS,Chipset,Card slot,...,RAM,Num of MC,Pic MC,Video MC,Num of SC,Pic SC,Loudspeaker,3.5mm jack,Battery,Price
1253,3365,Bold N2,BLU,"2022, August 11","Available. Released 2022, August 11","6.6 inches, 106.9 cm2 (~90.9% screen-to-body r...","1080 x 2340 pixels, 19.5:9 ratio (~390 ppi den...",Android 11,Mediatek MT6833P Dimensity 810 (6 nm),Unspecified,...,8.0,4.0,"64,5,2,2",1080p@30fps,2.0,"16,2",Yes,No,4200.0,About 230 EUR
1390,4007,C12,Nokia,"2023, January 18","Available. Released 2023, February 02","6.3 inches, 95.8 cm2 (~80.3% screen-to-body ra...","720 x 1600 pixels, 20:9 ratio (~278 ppi density)",Android 12 (Go edition),Unisoc SC9863A1 (28nm),Unspecified,...,2.0,1.0,8,Yes,1.0,5.0,Yes,Yes,3000.0,About 90 EUR
1562,4689,A97,Oppo,"2022, July 12","Available. Released 2022, July 15","6.6 inches, 104.9 cm2","1080 x 2408 pixels, 20:9 ratio (~400 ppi density)","Android 12, ColorOS 12",Mediatek MT6833P Dimensity 810 (6 nm),Unspecified,...,12.0,2.0,"48,2",1080p@30fps,1.0,12,"Yes, with stereo speakers",Yes,5000.0,About 310 EUR
1600,4727,A54s,Oppo,"2021, October 27","Available. Released 2021, November 18","6.52 inches, 102.6 cm2 (~82.9% screen-to-body ...","720 x 1600 pixels, 20:9 ratio (~269 ppi density)","Android 11, ColorOS 11.1",Mediatek MT6765G Helio G35 (12 nm),Unspecified,...,4.0,3.0,"50,2,2",1080p@30fps,1.0,8,Yes,Yes,5000.0,$ 199.99 / € 186.80 / £ 199.99 / C$ 243.50
1968,5751,Q3t,Realme,"2021, November 10","Available. Released 2021, November 15","6.6 inches, 104.8 cm2","1080 x 2412 pixels, 20:9 ratio (~400 ppi density)","Android 11, Realme UI 2.0",Qualcomm SM7325 Snapdragon 778G 5G (6 nm),Unspecified,...,8.0,3.0,"48,2,2","4K@30fps, 1080p@30/60fps",1.0,16,Yes,Yes,5000.0,About 290 EUR
2023,5812,V3,Realme,"2020, September 01","Available. Released 2020, September 10","6.5 inches, 102.0 cm2 (~81.6% screen-to-body r...","720 x 1600 pixels, 20:9 ratio (~270 ppi density)","Android 10, Realme UI",Mediatek MT6853V Dimensity 720 (7 nm),Unspecified,...,6.0,3.0,"13,2,2","4K@30/60fps, 1080p@30/60fps",1.0,8,Yes,Yes,5000.0,About 120 EUR
2138,6092,V11,Doogee,"2022, July 15","Available. Released 2022, July","6.39 inches, 100.2 cm2 (~72.9% screen-to-body ...","720 x 1560 pixels, 19.5:9 ratio (~269 ppi dens...",Android 11,Mediatek MT6833 Dimensity 700 (7 nm),Unspecified,...,8,3.0,"48,8,2","1440p, 1080p",1.0,16,Yes,No,8500,About 350 EUR
2148,6105,V10,Doogee,"2021, August 02","Available. Released 2021, August","6.39 inches, 100.2 cm2 (~72.9% screen-to-body ...","720 x 1560 pixels, 19.5:9 ratio (~269 ppi dens...",Android 11,Mediatek MT6833 Dimensity 700 (7 nm),Unspecified,...,8,3.0,"48,8,2","1440p, 1080p",1.0,16,Yes,No,8500,About 350 EUR
2207,6338,G13 Pro,Gionee,"2022, January 28","Available. Released 2022, January 30","6.26 inches, 94.6 cm2 (~78.8% screen-to-body r...","720 x 1600 pixels, 20:9 ratio (~280 ppi density)",HarmonyOS,Unisoc Tiger T310 (12nm),Unspecified,...,4,2.0,"13,2",1080p@30fps,1.0,5,Yes,Yes,3500,About 80 EUR
2341,6572,Play5 5G,Honor,"2021, May 18","Available. Released 2021, May 26","6.53 inches, 102.9 cm2 (~85.2% screen-to-body ...","1080 x 2400 pixels, 20:9 ratio (~403 ppi density)","Android 10, Magic UI 4.0, no Google Play Services",Mediatek MT6853 Dimensity 800U (7 nm),Unspecified,...,8,4.0,"64,8,2,2","4K@30fps, 1080p@30fps, gyro-EIS",1.0,16,Yes,No,3800,About 270 EUR


Có thể thấy đa phần là các sản phẩm trong những năm gần đây. Trong khi gần đây các hãng điện thoại cũng bỏ dần khe cắm thẻ nhớ. Vì vậy sẽ replace giá trị này thành No

In [18]:
df[df['Card slot'] =='Yes, up to 64 GB (dedicated slot)']

,ID,Name,Brand,Announced,Status,Size,Resolution,OS,Chipset,Card slot,...,RAM,Num of MC,Pic MC,Video MC,Num of SC,Pic SC,Loudspeaker,3.5mm jack,Battery,Price
3510,9304,Ascend Mate,Huawei,"2013, January. Released 2013, March",Discontinued,"6.1 inches, 102.6 cm2 (~73.2% screen-to-body r...","720 x 1280 pixels, 16:9 ratio (~241 ppi density)","Android 4.1 (Jelly Bean), upgradable to 4.2.2 ...",Huawei K3V2,"Yes, up to 64 GB (dedicated slot)",...,2.0,1.0,8,1080p@30fps,1.0,1,Yes,Yes,4050.0,About 230 EUR


Loại thẻ nhớ mà điện thoại này hỗ trợ là microSD 

Vậy cần giảm bớt thông tin chi tiết của loại thẻ nhớ hỗ trợ

### OS

In [19]:
df.OS.value_counts()

Android 4.4.2 (KitKat)                                                 171
Android 10                                                             138
Android 11                                                             116
Android 4.2 (Jelly Bean)                                               115
Android 6.0 (Marshmallow)                                              109
                                                                      ... 
Android 4.4.4 (KitKat), EMUI 2.3                                         1
Android 4.2.2 (Jelly Bean), EMUI Lite 2                                  1
Android 4.2.2 (Jelly Bean), EMUI 1.6                                     1
HP webOS 2.2                                                             1
Android 2.3.6 (Gingerbread), upgradable to 4.0 (Ice Cream Sandwich)      1
Name: OS, Length: 1194, dtype: int64

Có 1194 phiên bản hệ điều hành khác nhau

In [20]:
os = df.OS.unique()
lst_apk = []
lst_mcs = []
lst_padOS = []
lst_ios = []
lst_not_apk=[]
idx = 0
for o in df.OS:
    s = o.lower()
    if 'android' in s:
        lst_apk.append(idx)
    elif 'microsoft' in s: 
        lst_mcs.append(idx)
    elif 'ipados' in s:
        lst_padOS.append(idx)
    elif 'ios' in s:
        lst_ios.append(idx)
    else:
        lst_not_apk.append(idx)
    idx += 1
len(lst_apk), len(lst_mcs),len(lst_ios), len(lst_padOS), len(lst_not_apk)

(4655, 97, 47, 21, 64)

Đa phần đều là điện thoại hệ điều hành Android, sau đó là Microsoft, IOS, ipadOS,...

In [21]:
df.OS.loc[lst_not_apk].value_counts()

HarmonyOS 2.0                                                                11
HarmonyOS 3.0                                                                 4
HarmonyOS 2.0 (China), EMUI 12 (Europe), no Google Play Services              4
BlackBerry OS 10, upgradable to 10.3.1                                        2
EMUI 13 (International); HarmonyOS 3.0 (China)                                2
HarmonyOS                                                                     2
Symbian OS 9.2, Series 60 v3.1 UI                                             2
EMUI 12, no Google Play Services                                              2
BlackBerry OS 10.2, upgradable to 10.3.1                                      2
Tizen 2.4                                                                     2
Sailfish 2.0                                                                  2
EMUI 12                                                                       1
Tizen 2.3, upgradable to 2.4            

 Có thêm các hệ điều hành: 
- HarmonyOS, EMUI (hãng Huawei)
- MIUI 13 (Xiaomi, dựa trên Android 12)
- BlackBerry OS (BlackBerry)
- Symbian, Maemo  (Nokia, đã dừng phát triển)
- Sailfish, Firefox, MeeGo, Chrome  (dựa trên Linux)
- Bada (đã dừng phát triển, thay thế bởi Tizen)

Cần sửa về format: <tên hệ điều hành> <phiên bản>

Lược bớt 1 vài thông tin chi tiết

### Chipset

In [22]:
df.Chipset.value_counts()

Qualcomm MSM8916 Snapdragon 410 (28 nm)                                         94
Mediatek MT6833 Dimensity 700 (7 nm)                                            76
Mediatek MT6762 Helio P22 (12 nm)                                               75
Mediatek MT6582 (28 nm)                                                         65
Qualcomm MSM8939 Snapdragon 615 (28 nm)                                         59
                                                                                ..
Qualcomm MSM8270                                                                 1
Qualcomm MSM8936 Snapdragon 610 (28 nm)                                          1
Mediatek MT6582 - 3G modelQualcomm MSM8916 Snapdragon 410 (28 nm) - 4G model     1
Mediatek MT6755 (28 nm) or MTK7650 (28 nm)                                       1
Nvidia Tegra 2                                                                   1
Name: Chipset, Length: 703, dtype: int64

Chipset mang thông tin hãng, mã sản phẩm, loại, phiên bản, tiến trình xử lý

Một số hãng chip như: Mediatek, Qualcomm, Samsung (Exynos tiền thân là hummingbird), HiSilicon (kirin), Apple (apple), unisoc(tiger,Spreadtrum), Intel(intel), nvidia(tegra), Texas Instruments (OMAP)

In [23]:
mt=[]
qc=[]
ss=[]
his=[]
ap=[]
uni =[]
nvidia=[]
intel=[]
ti =[]
other = []
i = 0
for c in df.Chipset:
    s = c.lower()
    if 'mediatek' in s: mt.append(i)
    elif 'qualcomm' in s : qc.append(i)
    elif 'exynos' in s or 'hummingbird' in s: ss.append(i)
    elif 'apple' in s: ap.append(i)
    elif 'kirin' in s or 'hisilicon' in s: his.append(i)
    elif 'unisoc' in s or 'spreadtrum' in s or 'tiger' in s: uni.append(i)
    elif 'tegra' in s : nvidia.append(i)
    elif 'intel' in s: intel.append(i)
    elif 'omap' in s: ti.append(i)
    else: other.append(i)
    i += 1
print('Mediatek:',len(mt))
print('Qualcomm:',len(qc))
print('Samsung:',len(ss))
print('HiSilicon:',len(his))
print('Apple:',len(ap))
print('Unisoc:',len(uni))
print('Nvidia:',len(nvidia))
print('Intel:',len(intel))
print('TI OMAP:',len(ti))
print('Other:',len(other))

Mediatek: 1827
Qualcomm: 2185
Samsung: 181
HiSilicon: 185
Apple: 65
Unisoc: 191
Nvidia: 58
Intel: 67
TI OMAP: 62
Other: 63


Ngoài ra còn một vài chip điện tử khác

In [24]:
df.Chipset.loc[other].value_counts()

Huawei K3V2                10
NovaThor U8500              6
Broadcom BCM23550           5
Marvell PXA1088             5
Google Tensor (5 nm)        3
Marvell PXA986              3
NovaThor U8420              2
Rockchip RK3399             2
Allwinner A31               2
Rockchip 3066               2
Rockchip RK3188             2
BCM23550                    2
Google Tensor G2 (5 nm)     2
Huawei K3V2E                1
Broadcom BC28155            1
Leadcore L1860C             1
Xiaomi Surge S1             1
JLQ JR510 (11 nm)           1
Nvidia SL460N               1
Marvell 88sv331             1
Unspecified                 1
Pega-Dual +XMM6262          1
LG Nuclun                   1
BCM 28145/28155             1
Marvell Armada PXA1908      1
Chipset                     1
ATI Graphic Chip W2284      1
Rockchip RK101              1
Broadcom BCM2727            1
Allwinner A133              1
Name: Chipset, dtype: int64

### Pic MC, SC

In [25]:
df['Pic MC'].value_counts()

8              773
13             757
5              605
13,2           201
16             167
              ... 
50,64,64,64      1
50,64,64,13      1
40,8             1
40,8,2           1
24,5             1
Name: Pic MC, Length: 369, dtype: int64

In [26]:
df['Pic SC'].value_counts()

5         959
8         868
16        600
2         529
0.3       508
         ... 
2,3         1
13,4        1
4,13        1
Pic SC      1
2,1         1
Name: Pic SC, Length: 95, dtype: int64

Có thể có nhiều camera nên có nhiều giá trị trong một bản ghi. Kiểm tra thêm trong phần Logic

### Video MC

In [27]:
df['Video MC'].value_counts()

1080p@30fps                                              2178
720p@30fps                                                358
Yes                                                       350
4K@30fps, 1080p@30fps                                     218
720p                                                      134
                                                         ... 
4K@30fps, 1080p@30/60/120fps (gyro-EIS)                     1
4K@30fps, 1080p@60/120fps, 720p@240fps                      1
4K@15fps, 1080p@30fps                                       1
4K@30/60fps, 1080p@30/30/60/120fps, stereo sound rec.       1
352x288                                                     1
Name: Video MC, Length: 434, dtype: int64

Có nhiều chất lượng video và tốc độ khung hình mỗi giây khác nhau. Ưu tiên chọn độ phân giải quay video cao nhất và tốc độ khung hình quay cao nhất. Tuy nhiên cần tách thành 2 bảng khác nhau do 2 thuộc tính cao nhất này không cùng chế độ quay

In [28]:
video = df['Video MC']
res = []
fps = []
i=-1
for s in video:
    i+=1
    if s == 'Yes':
        continue
    lst = s.split(', ')
    best_res = lst[0].split('@')[0]
    max_fps = 0
    for q in lst:
        lst_tmp = q.split('@')
        if len(lst_tmp) > 1:
            ss = lst_tmp[1]
            if 'fps' in ss:
                ss = ss[:-3]
            max_fps = max(max_fps, np.max(np.array(re.findall("\d+", ss)).astype(int)))
    res.append(best_res)
    fps.append(max_fps)
len(res), len(fps), len(df)

(4534, 4534, 4884)

Còn 350 giá trị Yes

In [29]:
df['Pic MC'][df['Video MC']=='Yes'].value_counts()

5           141
8            96
2            38
15           29
13           24
16            4
0.3           3
5,8           3
12            2
5,2           2
3             2
48,5,2,2      1
12,2          1
8,2           1
48,13,2       1
21            1
64,8,50       1
Name: Pic MC, dtype: int64

In [30]:
df[(df['Video MC']=='Yes') & (df['Pic MC'] == '64,8,50')]

,ID,Name,Brand,Announced,Status,Size,Resolution,OS,Chipset,Card slot,...,RAM,Num of MC,Pic MC,Video MC,Num of SC,Pic SC,Loudspeaker,3.5mm jack,Battery,Price
4697,11685,nubia Z40 Pro,ZTE,"2022, February 25","Available. Released 2022, March 02","6.67 inches, 107.4 cm2 (~91.0% screen-to-body ...","1080 x 2400 pixels, 20:9 ratio (~395 ppi density)","Android 12, MyOS 12",Qualcomm SM8450 Snapdragon 8 Gen 1 (4 nm),No,...,8.0,3.0,"64,8,50",Yes,1.0,16,"Yes, with stereo speakers",No,5000.0,About 500 EUR


- Có những dòng máy cao cấp nhưng không có thông tin chất lượng video.
- Chất lượng video cũng phụ thuộc chủ yếu vào chất lượng chụp ảnh.
- Có thể thấy các máy có Pic of MC cao sẽ có chất lượng quay phim 8k
- Các máy 0.3MP

### Ram, rom

In [31]:
df.RAM.value_counts()

1.0     688
4.0     682
2.0     660
8.0     480
3.0     454
6.0     407
0.0     231
2       196
1       195
4       186
3       177
6       121
0       120
8        96
5        53
12.0     45
5.0      28
12       23
16.0      9
11        5
10.0      3
18.0      3
9.0       3
32        2
13        2
13.0      2
26        1
RAM       1
14        1
9         1
26.0      1
16        1
45.0      1
77.0      1
22.0      1
25.0      1
85.0      1
32.0      1
7.0       1
Name: RAM, dtype: int64

In [32]:
df[df.RAM=='85.0']

,ID,Name,Brand,Announced,Status,Size,Resolution,OS,Chipset,Card slot,...,RAM,Num of MC,Pic MC,Video MC,Num of SC,Pic SC,Loudspeaker,3.5mm jack,Battery,Price
494,1117,P700i,Lenovo,"2012. Released 2013, January",Discontinued,"4.0 inches, 45.5 cm2 (~56.1% screen-to-body ra...","480 x 800 pixels, 5:3 ratio (~233 ppi density)",Android 4.0.4 (Ice Cream Sandwich),Mediatek MT6577 (40 nm),"microSDHC (dedicated slot), 2 GB included",...,85.0,1.0,5,720p@30fps,1.0,0.3,Yes,Yes,2500.0,About 140 EUR


- Ram xuất hiện nhiều giá trị bất thường, không đúng so với thông tin của máy
- Xuất hiện 351 giá trị bằng 0.

In [33]:
df.ROM.value_counts()

128.0    822
16.0     679
64.0     671
32.0     596
8.0      390
16       292
0.0      231
32       210
128      180
4.0      178
64       172
8        127
256.0    121
0        120
256       40
4         29
512.0     12
512        7
2.0        2
258.0      1
1          1
ROM        1
11         1
2          1
Name: ROM, dtype: int64

In [34]:
df[df.ROM=='11']

,ID,Name,Brand,Announced,Status,Size,Resolution,OS,Chipset,Card slot,...,RAM,Num of MC,Pic MC,Video MC,Num of SC,Pic SC,Loudspeaker,3.5mm jack,Battery,Price
3237,8674,Xperia acro HD SO-03D,Sony,"2012, May. Released 2012, September",Discontinued,"4.3 inches, 51.0 cm2 (~61.3% screen-to-body ra...","720 x 1280 pixels, 16:9 ratio (~342 ppi density)",Android 2.3 (Gingerbread),Qualcomm MSM8260 Snapdragon S3,microSDHC (dedicated slot),...,1,1.0,12,1080p@30fps,1.0,3,Yes,Yes,1910,About 290 EUR


- Các giá trị của ROM chính xác hơn RAM, ngoài ra cũng xuất hiện 351 giá trị 0

RAM và ROM có thể cần cào lại để thay thế 351 giá trị 0. Tuy nhiên một số máy có thông tin của ROM nhưng không có thông tin của RAM, vì vậy có thể đã mặc định gán cả 2 đều bằng 0. Vậy nên các giá trị này có thể loại bỏ do thiếu thông tin của RAM. Chỉ cần cào lại giá trị chính xác của RAM (giá trị khác 0)

### Loudspeaker

In [35]:
df.Loudspeaker.value_counts()

Yes                                                                 3778
Yes, with stereo speakers                                            901
Yes, with dual speakers                                               71
Yes, with stereo speakers (4 speakers)                                70
Yes, dual speakers                                                    24
Yes, with stereo speakers (2 amplifiers)                               8
Yes, with DTS:X stereo speakers (2 dedicated amplifiers)               6
Yes, with stereo speakers (8 speakers)                                 4
Yes, dual mono speakers                                                3
Yes, built-in amplifier                                                2
Yes, with Dolby Atmos stereo speakers (THX-certified amplifiers)       2
Yes, with Dolby Atmos stereo speakers (2 dedicated amplifiers)         2
Yes, (unconfirmed stereo speakers)                                     1
Loudspeaker                                        

In [36]:
df[df.Loudspeaker=='No']

,ID,Name,Brand,Announced,Status,Size,Resolution,OS,Chipset,Card slot,...,RAM,Num of MC,Pic MC,Video MC,Num of SC,Pic SC,Loudspeaker,3.5mm jack,Battery,Price
3870,10296,G910 / G920,Toshiba,"2008, January. Released 2008, June",Discontinued,"3.0 inches, 25.6 cm2 (~34.2% screen-to-body ra...","800 x 480 pixels (Wide-VGA), 5:3 ratio (~311 p...",Microsoft Windows Mobile 6 Professional,Qualcomm MSM7200,microSD (dedicated slot),...,0.0,1.0,2,Yes,1.0,0.3,No,No,1320.0,About 190 EUR


Trừ giá trị là *No* và 1 giá trị bất thường, còn lại đều là *Yes*? Có thể loại bỏ cột này và 2 bản ghi có giá trị khác *Yes*

### 3.5mm jack

In [37]:
df['3.5mm jack'].value_counts()

Yes                               4181
No                                 695
Unspecified                          6
No, included adaptor for 3.5mm       1
3.5mm jack                           1
Name: 3.5mm jack, dtype: int64

Thuộc tính này thể hiện thiết bị có jack cắm tai nghe 3.5mm hay không. 

Có một bản ghi là không có jack 3.5mm nhưng lại đi kèm bộ chuyển đổi vì thế ta sẽ coi như thực thể này có jack cắm 3.5mm

Ngoài ra có một bản ghi là '3.5mm jack' có thể do khi nối các bản dữ liệu khi cào bị như vậy vì thế ta sẽ xóa bản ghi này đi

In [38]:
df.Announced[df['3.5mm jack'] == 'Unspecified'].unique()

array(['2023, January 03', '2022, May 27', '2022, March 18',
       '2021, November 27', '2021, July 06', '2021, April'], dtype=object)

Các máy đều là các mẫu điện thoại trong những năm gần đây. Mà gần đây các mẫu điện thoại dần loại bỏ jack tai nghe 3.5mm. Vì vậy có thể thay thế giá trị này thành No

### Price

In [39]:
df.Price.value_counts()

About 200 EUR      235
About 150 EUR      210
About 130 EUR      195
About 100 EUR      182
About 120 EUR      176
                  ... 
$ 1,190.00           1
₹ 12,490             1
About 11000 INR      1
About 1350 EUR       1
$ 890.00             1
Name: Price, Length: 759, dtype: int64

Có nhiều đơn vị tiền tệ và format khác nhau

In [40]:
for s in df.Price:
    l = s.split(' ')
    if len(l) == 3:
        if l[0].lower() == 'about' and l[-1].lower() == 'eur':
            continue
    print(s)

$ 939.95 / € 1,275.99 / £ 1,020.00 / ₹ 127,999
$ 999.99 / € 1,164.99 / £ 1,039.00 / ₹ 120,900
$ 719.99 / € 1,004.00 / £ 894.00 / ₹ 80,999
$ 769.00 / € 879.01 / £ 699.00 / ₹ 71,999
$ 198.99 / € 489.00 / £ 449.00 / ₹ 43,900
$ 959.00 / € 1,505.34 / £ 1,299.00 / Rp 23,681,736
$ 892.00 / € 1,430.44 / £ 1,249.00 / ₹ 142,999 / Rp 19,390,000
$ 729.99 / € 793.94 / £ 599.00 / ₹ 60,900 / C$ 919.00 / Rp 16,614,000
$ 634.95 / € 719.00 / £ 596.96 / ₹ 89,999 / Rp 14,566,220
$ 468.04 / Rp 20,299,000
$ 575.00 / € 1,575.39 / ₹ 60,800 / C$ 1,104.45
$ 394.95 / € 699.00 / £ 599.00 / ₹ 50,900 / Rp 12,275,000
$ 325.00 / € 700.28 / £ 499.00 / Rp 10,285,000
$ 164.38 / € 189.00 / £ 279.99
$ 424.80 / C$ 755.00
$ 449.19 / € 418.90 / £ 939.00 / Rp 9,999,000
$ 260.00 / € 499.99 / £ 257.00 / Rp 4,809,000
$ 655.30 / € 1,349.00 / £ 1,238.00 / C$ 2,099.99
$ 698.99 / £ 809.99 / C$ 1,499.99
$ 1,099.99 / € 1,299.00
$ 999.99 / € 1,457.26 / £ 1,099.99
$ 599.99 / € 749.00 / £ 709.99
$ 1,480.00
$ 627.99 / € 1,061.00 / £ 829.9

Có thể thấy còn đơn vị INR, Rp và một vài đơn vị tiền tệ khác bị xen lẫn, cần đổi các giá trị này sang cùng 1 đơn vị tiền tệ là Euro

<font size = '3'> **Vì các giá trị đang là object và đang chưa nhiều thông tin nhiễu nên chưa thể kiểm tra toàn bộ vấn đề về outliers và Logic** <font>

## Giá trị bất thường

### Rom, Ram
Xuất hiện các giá trị bằng 0

In [41]:
len(df[df.ROM == '0.0']), len(df[df.RAM == '0.0']), len(df[df.ROM == '0']), len(df[df.RAM == '0'])

(231, 231, 120, 120)

Có thể trên cùng bản ghi

In [42]:
df_RR = df[((df.ROM == '0.0')|(df.ROM == '0')) & ((df.RAM == '0.0')|(df.RAM == '0'))]
len(df_RR)

351

- Nguyên nhân do không có giá trị RAM nên mặc định chuyển cả 2 giá trị đều thành 0

## Logic

### Num of MC/SC và Pic of MC/SC
Số camera và độ phân giải tương ứng của từng camera. Số giá trị độ phân giải phải bằng số camera

#### MC

In [43]:
df['Pic MC'].value_counts()

8              773
13             757
5              605
13,2           201
16             167
              ... 
50,64,64,64      1
50,64,64,13      1
40,8             1
40,8,2           1
24,5             1
Name: Pic MC, Length: 369, dtype: int64

In [44]:
df['Num of MC'].value_counts()

1.0          2730
3.0           966
2.0           748
4.0           439
Num of MC       1
Name: Num of MC, dtype: int64

In [45]:
error =[]
for i in range(len(df)):
    lst_pic = df.at[i,'Pic MC'].split(',')
    n = df.at[i,'Num of MC']
    if n == 'Num of MC':
        continue
    n = int(n[:-2])
    if len(lst_pic) != n:
        error.append(i)
len(error)

281

In [46]:
df['Num of MC'].loc[error].value_counts()

3.0    91
1.0    82
2.0    74
4.0    34
Name: Num of MC, dtype: int64

In [47]:
df.loc[error][df.loc[error]['Num of MC'] == '3.0']

,ID,Name,Brand,Announced,Status,Size,Resolution,OS,Chipset,Card slot,...,RAM,Num of MC,Pic MC,Video MC,Num of SC,Pic SC,Loudspeaker,3.5mm jack,Battery,Price
709,1874,18s Pro,Meizu,"2021, September 22","Available. Released 2021, September 23","6.7 inches, 107.6 cm2 (~90.7% screen-to-body r...","1440 x 3200 pixels, 20:9 ratio (~526 ppi density)","Android 11, Flyme 9.2",Qualcomm SM8350 Snapdragon 888+ 5G (5 nm),No,...,8.0,3.0,"50,8,32,3","8K, 4K@30/60fps, 1080p; gyro-EIS, HDR",1.0,44,"Yes, with stereo speakers",No,4500.0,About 610 EUR
712,1878,18 Pro,Meizu,"2021, March 03","Available. Released 2021, March 08","6.7 inches, 107.6 cm2 (~90.7% screen-to-body r...","1440 x 3200 pixels, 20:9 ratio (~526 ppi density)","Android 11, Flyme 9",Qualcomm SM8350 Snapdragon 888 5G (5 nm),No,...,8.0,3.0,"50,8,32,3","8K, 4K@30/60fps, 1080p; gyro-EIS",1.0,44,"Yes, with stereo speakers",No,4500.0,About 640 EUR
714,1880,17 Pro,Meizu,"2020, May 08","Available. Released 2020, May 11","6.6 inches, 106.9 cm2 (~86.6% screen-to-body r...","1080 x 2340 pixels, 19.5:9 ratio (~390 ppi den...","Android 10, Flyme 8.1",Qualcomm SM8250 Snapdragon 865 5G (7 nm+),No,...,8.0,3.0,"64,8,32,3","4K@30/60fps, 1080p; gyro-EIS",1.0,20,"Yes, with stereo speakers",No,4500.0,About 560 EUR
718,1884,16s Pro,Meizu,"2019, August","Available. Released 2019, August","6.2 inches, 95.8 cm2 (~85.9% screen-to-body ra...","1080 x 2232 pixels, 18.5:9 ratio (~403 ppi den...","Android 9.0 (Pie), Flyme 8",Qualcomm SM8150 Snapdragon 855+ (7 nm),No,...,6.0,3.0,"48,16","4K@30/60fps, 1080p@30fps (gyro-EIS)",1.0,0.3,Yes,No,3600.0,About 340 EUR
1216,3323,A55 Pro,Blackview,"2022, March 07","Available. Released 2022, March","6.53 inches, 102.9 cm2 (~81.4% screen-to-body ...","720 x 1600 pixels, 20:9 ratio (~269 ppi density)",Android 11,Mediatek MT6762V Helio P22 (12 nm),microSDXC,...,4.0,3.0,13,1080p@30fps,1.0,5,Yes,Yes,4780.0,About 130 EUR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4276,11038,Power U20,Wiko,"2021, June","Available. Released 2021, June","6.82 inches, 104.8 cm2 (~76.7% screen-to-body ...",720 x 1640 pixels (~270 ppi density),Android 11,Mediatek MT6765G Helio G35 (12 nm),microSDXC,...,3.0,3.0,"13,2",1080p@30fps,1.0,5,Yes,Yes,6000.0,About 120 EUR
4371,11208,Poco X4 Pro 5G,Xiaomi,"2022, February 28","Available. Released 2022, March 23","6.67 inches, 107.4 cm2 (~86.0% screen-to-body ...","1080 x 2400 pixels, 20:9 ratio (~395 ppi density)","Android 11, MIUI 13 for POCO",Qualcomm SM6375 Snapdragon 695 5G (6 nm),microSDXC (uses shared SIM slot),...,6.0,3.0,"108,64,8,2",1080p@30fps,1.0,16,"Yes, with stereo speakers",Yes,5000.0,"$ 262.00 / € 311.38 / £ 288.89 / ₹ 17,879"
4645,11537,Max 3 Plus,Yezz,"2022, October","Available. Released 2023, January","6.1 inches, 93.1 cm2 (~75.4% screen-to-body ra...","480 x 1010 pixels, 19:9 ratio (~183 ppi density)",Android 12,Unisoc SC9832E (28nm),microSDXC,...,2.0,3.0,"13,3",1080p@30fps,1.0,8,Yes,Yes,3700.0,About 90 EUR
4646,11538,Max 3,Yezz,"2022, September","Available. Released 2023, January","5.45 inches, 76.7 cm2 (~66.5% screen-to-body r...","720 x 1440 pixels, 18:9 ratio (~295 ppi density)",Android 12,Unisoc SC9832E (28nm),microSDXC,...,2.0,3.0,"13,3",1080p@30fps,1.0,8,Yes,Yes,2600.0,About 70 EUR


- Số giá trị ở Pic nhỏ hơn bởi một số camera sau phụ không có thông tin. Phương pháp xử lý có thể điền thêm giá trị cũ cho đủ thông tin 
- Số giá trị ở Pic nhiều hơn so với số camera sau vì có thêm chế độ làm độ phân giải giảm đi, chẳng hạn như việc xoá phông. Có 2 trường hợp sau: 
    + Với các trường hợp bị lặp nhưng độ phân giải giữ nguyên, ta chỉ lấy 1 độ phân giải duy nhất
    + Bỏ đi các độ phân giải thấp hơn

#### SC

In [48]:
df['Num of SC'].value_counts()

1.0          4740
2.0           135
3.0             8
Num of SC       1
Name: Num of SC, dtype: int64

Không có sản phẩm nào có 4 camera trước. Tuy nhiên vẫn có 8 sản phẩm có 3 camera trước

In [49]:
df.Name[df['Num of SC'] =='3.0']

219       Zenfone 8 Flip
224        Zenfone 7 Pro
225            Zenfone 7
2391             Magic 2
2766          Galaxy A80
4174    NEX Dual Display
4489        Mi Mix Alpha
4746           nubia Z20
Name: Name, dtype: object

Tất cả đều là sản phẩm 2 màn hình hoặc tràn viền 180% (đã trình bày)

In [50]:
error =[]
for i in range(len(df)):
    lst_pic = df.at[i,'Pic SC'].split(',')
    n = df.at[i,'Num of SC']
    if n == 'Num of SC':
        continue
    n = int(n[:-2])
    if len(lst_pic) != n:
        error.append(i)
len(error)

64

In [51]:
df[['Num of SC','Pic SC']].loc[error]

,Num of SC,Pic SC
10,1.0,"5,8"
191,1.0,"5,5"
237,1.0,"8,16"
240,2.0,20
241,1.0,"8,13"
...,...,...
4771,1.0,"5,5"
4786,1.0,"8,2"
4794,1.0,"5,13"
4834,1.0,"5,2"


Vấn đề và phương pháp xử lý cũng tương tự với MC

# Làm sạch dữ liệu

Xoá bỏ dòng trùng với tên cột

In [52]:
df.drop(df[df.Name == 'Name'].index, inplace=True)
df.shape

(4883, 21)

Đã xoá đi 1 bản ghi bị trùng tên cột

In [53]:
dfnew = df.copy()
idf = dfnew.index

## Announced, Status
loại bỏ các giá trị Released. Status chuyển về 2 giá trị duy nhất

### Status

In [54]:
for i in idf:
    dfnew.at[i,'Status'] = dfnew.at[i,'Status'].split('.')[0]

In [55]:
dfnew.Status.value_counts()

Available       2710
Discontinued    2143
Cancelled         21
Coming soon        9
Name: Status, dtype: int64

Dữ liệu cột ``Status`` đã được làm sạch

### Announced
- Đưa về dạng datetime tháng/năm

In [56]:
i_not_A = []
i_year = []
for i in idf:
    s = dfnew.at[i,'Announced'].split('.')[0]
    if s == 'Not announced yet':
        i_not_A.append(i)
    if len(s) == 4:
        i_year.append(i)

In [57]:
print(len(i_year),'bản ghi chỉ có năm, không có tháng')

66 bản ghi chỉ có năm, không có tháng


Ta có thể mặc định gán một giá trị bất kì trong 12 tháng có thể là tháng 1

In [58]:
print(len(i_not_A),'không phải định dạng ngày tháng')

4 không phải định dạng ngày tháng


In [59]:
dfnew[['Brand','Name','Announced','Status']].loc[i_not_A]

,Brand,Name,Announced,Status
3098,Samsung,Galaxy A72 5G,Not announced yet,Cancelled
4594,Xiaomi,Redmi 11,Not announced yet,Cancelled
4595,Xiaomi,Redmi 20X,Not announced yet,Cancelled
4596,Xiaomi,Mi 10 Lite Zoom,Not announced yet,Cancelled


- Tất cả đều là chưa công bố chính thức và bị huỷ. Tuy nhiên có thể tìm kiếm thông tin và thay thế lại.
- Samsung Galaxy A72 5G được công bố vào ngày 17 tháng 3 năm 2021
- Xiaomi Redmi 11 được công bố vào ngày 25 tháng 8 năm 2021
- Xiaomi Redmi 20X vẫn chưa được công bố chính thức, nhưng dự kiến ra mắt tại Ấn Độ vào 30 tháng 4 năm 2021
- Mi 10 Lite Zoom được công bố vào ngày 27 tháng 4 năm 2020.

In [60]:
dfnew.at[3098,'Announced'] = '2021, March'
dfnew.at[4594,'Announced'] = '2021, August'
dfnew.at[4595,'Announced'] = '2021, April'
dfnew.at[4596,'Announced'] = '2020, April'

In [61]:
month = {'january':1,'february':2,'march':3,'april':4,
        'may':5, 'june':6, 'july':7,'august':8,
        'september':9,'october':10,'november':11,'december':12,
        'q1':1,'q2':4,'q3':7,'q4':10}

In [62]:
for i in idf:
    s = dfnew.at[i,'Announced'].split('. Released')[0]
    lst = s.split(', ')
    y = int(re.findall("\d+", lst[0])[0])
    m = 1
    if len(lst) != 1:
        ss = lst[1].lower().split(' ')[0]
        m = month[ss]
    # Mặc định ngày bằng 1, tuy nhiên không cần lấy thông tin ngày
    dfnew.at[i,'Announced'] = datetime.date(y,m,1)

In [63]:
dfnew.Announced.value_counts()

2020-09-01    78
2021-03-01    76
2014-01-01    72
2014-09-01    70
2021-04-01    68
              ..
2005-01-01     1
2005-04-01     1
2009-08-01     1
2009-03-01     1
2008-01-01     1
Name: Announced, Length: 178, dtype: int64

## Size
Có thể lấy 2 thông tin là kích thước màn hình (inches) và diện tích màn hình (area)

In [64]:
dfnew_2 = dfnew.copy()

In [65]:
area = []
for i in idf:
    # Lấy thông tin số trong chuỗi
    inc,s = dfnew_2.at[i,'Size'].split(' inches')
    a = re.findall(r"(\d*\.*\d) cm2", s)[0]
    dfnew_2.at[i,'Size'] = float(inc)
    area.append(float(a))
len(area)

4883

In [66]:
dfnew_2['ScreenArea'] = area

In [67]:
dfnew_2['Size'] = dfnew_2['Size'].astype(float)

## Resolution

In [68]:
ppi = []
for i in idf:
    # Thay các giá trị có format khác
    if i == 584 or i == 2412: p,x= '1080 x 1920',386
    elif i == 864 or i == 869: p,x = '540 x 960',220
    elif i == 888: p,x='320 x 480',156 
    else:   
        p,s = dfnew_2.at[i,'Resolution'].split(' pixels')
        x = re.findall(r"(\d*\.*\d) ppi density", s)[0]
    dfnew_2.at[i,'Resolution'] = p
    ppi.append(float(x))
len(ppi)

4883

In [69]:
dfnew_2['ppi'] = ppi

## Card slot

In [70]:
df_3 = dfnew_2.copy()

In [71]:
lst_card = 'SD,miniSD,microSD,microSDHC,microSDXC,NM,RS-DV-MMC'.split(',')
lst_card

['SD', 'miniSD', 'microSD', 'microSDHC', 'microSDXC', 'NM', 'RS-DV-MMC']

Thay đổi giá trị tìm hiểu được

In [72]:
df_3.at[df_3[df_3['Card slot']=='Yes, up to 64 GB (dedicated slot)'].index[0],'Card slot'] = 'microSD'

In [73]:
df_3['Card slot'].replace(value='No', to_replace = 'Unspecified', inplace=True)

In [74]:
for i in idf:
    card = df_3.at[i,'Card slot']
    card = card.translate({ord('/'): ' ', ord('.'):None,ord('('):None,ord(')'):None,ord(','):None}).split(' ')
    for c in lst_card:
        if c in card:
            df_3.at[i,'Card slot'] = c
            break

In [75]:
df_3['Card slot'].value_counts()

microSDXC    2774
microSDHC    1046
No            957
NM             65
microSD        34
SD              4
RS-DV-MMC       2
miniSD          1
Name: Card slot, dtype: int64

In [76]:
df_3[df.columns[6:]][:4]

,Resolution,OS,Chipset,Card slot,ROM,RAM,Num of MC,Pic MC,Video MC,Num of SC,Pic SC,Loudspeaker,3.5mm jack,Battery,Price
0,600 x 1024,Android 8.1 (Oreo),Mediatek MT8321A/D (28 nm),microSDXC,8.0,1.0,1.0,2,720p@30fps,1.0,0.3,Yes,Yes,2580.0,About 70 EUR
1,1080 x 1920,Android 7.1 (Nougat),Mediatek MT6757 Helio P20 (16 nm),microSDXC,32.0,3.0,1.0,12,1080p@30fps,1.0,8,"Yes, with stereo speakers",Yes,2850.0,About 140 EUR
2,1080 x 1920,Android 7.1 (Nougat),Mediatek MT6753 (28 nm),microSDXC,16.0,3.0,1.0,13,1080p@30fps,1.0,5,Yes,Yes,2800.0,About 120 EUR
3,1080 x 1920,Android 7.1 (Nougat),Mediatek MT6750S (28 nm),microSDHC,32.0,3.0,2.0,"12,2",1080p@30fps,1.0,5,Yes,Yes,4000.0,About 260 EUR


## Num of MC, Num of SC
Chuyển sang dạng số nguyên

In [77]:
df_3['Num of MC'].value_counts()

1.0    2730
3.0     966
2.0     748
4.0     439
Name: Num of MC, dtype: int64

In [78]:
df_3['Num of MC'].replace(value=[1,2,3,4], to_replace=['1.0','2.0','3.0','4.0'], inplace=True)

In [79]:
df_3['Num of SC'].value_counts()

1.0    4740
2.0     135
3.0       8
Name: Num of SC, dtype: int64

In [80]:
df_3['Num of SC'].replace(value=[1,2,3], to_replace=['1.0','2.0','3.0'], inplace=True)

In [81]:
df_3.dtypes

ID               int64
Name            object
Brand           object
Announced       object
Status          object
Size           float64
Resolution      object
OS              object
Chipset         object
Card slot       object
ROM             object
RAM             object
Num of MC        int64
Pic MC          object
Video MC        object
Num of SC        int64
Pic SC          object
Loudspeaker     object
3.5mm jack      object
Battery         object
Price           object
ScreenArea     float64
ppi            float64
dtype: object

## Loudspeaker và 3.5mm jack

### Loudspeaker
Chuyển về *Yes/No*. Tuy nhiên ``Loudspeaker`` chỉ có 1 giá trị là *No* trong tổng số hơn 4400 dữ liệu. Vậy có thể loại bỏ hàng có giá trị No và xoá cột ``Loudspeaker``

In [82]:
df_4 = df_3.drop(df_3[df_3.Loudspeaker =='No'].index[0]).drop(columns=['Loudspeaker'])
idf = df_4.index
print(df_4.shape)

(4882, 22)


### jack 3.5mm 
- Thay thế các giá trị *Unspecified* thành *No* do xu thế gần đây đã dần loại bỏ và không tìm được thông tin Yes/No 
- Thay giá trị *No, included adaptor for 3.5mm* thành *Yes* vì vẫn có bộ chuyển đổi

In [83]:
df_4['3.5mm jack'].value_counts()

Yes                               4181
No                                 694
Unspecified                          6
No, included adaptor for 3.5mm       1
Name: 3.5mm jack, dtype: int64

In [84]:
df_4['3.5mm jack'].replace(value=['No','Yes'], to_replace=['Unspecified','No, included adaptor for 3.5mm'], inplace=True)

In [85]:
df_4['3.5mm jack'].value_counts()

Yes    4182
No      700
Name: 3.5mm jack, dtype: int64

## Price
Chuyển đổi về cùng đơn vị tiền tệ là Eur, đưa về dạng số thực

In [86]:
data=df_4.copy()

In [87]:
FLoat = []
FLoat_index = []
String = []
String_index = []
for i in idf:
    if type(data.Price[i]) == float:
        FLoat.append(data.Price[i])
        FLoat_index.append(i)
    else :
        String.append(data.Price[i])
        String_index.append(i)

In [88]:
EUR = []
EUR_index = []
INR = []
INR_index = []
d = []
d_index = []
count = 0
idx = 0
for i in idf: 
    a = String[idx].split(" ")
    if len(a) == 3 and a[2] == 'EUR':
        EUR.append(a[1])
        EUR_index.append(String_index[idx])
    elif len(a) == 3 and a[2] == 'INR': 
        INR.append(a[1])
        INR_index.append(String_index[idx])
    else: 
        d.append(String[idx])
        d_index.append(String_index[idx])
    idx+=1

In [89]:
for i in range(len(EUR_index)): # CHUYỂN DẠNG ABOUT EUR THÀNH EUR
    data.Price.replace(to_replace=data.Price[EUR_index[i]], value= int(EUR[i]), inplace= True)

In [90]:
for i in range(len(INR_index)): # chuyển đổi từ INR sang eur với INR = 0.011 EUR
    data.Price.replace(to_replace=data.Price[INR_index[i]], value= (int(INR[i])*0.011), inplace= True)

In [91]:
Dollar = []
Dollar_index = []
Rup = []
Rup_index = []
Rp = []
Rp_index = []
EUR_1 = []
EUR_1_index = []
Bang = []
e = []
for i in range(len(d)):
    if d[i][0] == '$':
        Dollar.append(d[i].encode("ascii", "ignore").decode())
        Dollar_index.append(d_index[i])
    elif d[i][0] == '€' :
        EUR_1.append(d[i].encode("ascii", "ignore").decode())
        EUR_1_index.append(d_index[i])
    elif d[i][0] == '₹':
        Rup.append(d[i].encode("ascii", "ignore").decode())
        Rup_index.append(d_index[i])
    elif d[i][0] == '£':
        Bang.append(d[i].encode("ascii", "ignore").decode())
    elif d[i][0] == 'R':
        Rp.append(d[i].encode("ascii", "ignore").decode())
        Rp_index.append(d_index[i])
    else:
        e.append(d[i].encode("ascii", "ignore").decode())

In [92]:
for i in range(len(Dollar)): # chuyển từ dollar thành eur
    a = Dollar[i].split('/')[0].split('.')[0][1::]
    a = a.replace(',', '')
    data.Price.replace(to_replace= data.Price[Dollar_index[i]], value=(int(a)/(1.06)), inplace= True)

In [93]:
for i in range(len(EUR_1)): # chuyển nốt EUR
    a = EUR_1[i].split('/')[0].split('.')[0]
    a = a.replace(',','')
    data.Price.replace(to_replace=data.Price[EUR_1_index[i]], value=int(a), inplace=True)

In [94]:
for i in range(len(Rup)): # chuyển từ RUB sang EUR với gia 1 rub thành 0.012 EUR
    a = Rup[i].split('/')[0]
    a = a.replace(',','')
    data.Price.replace(to_replace=data.Price[Rup_index[i]], value= int(a)*0.012, inplace= True)

In [95]:
for i in range(len(Rp)):
    a = Rp[i][2::]
    a = a.replace(',', '')
#     print(int(a)*0.000061)
    data.Price.replace(to_replace=data.Price[Rp_index[i]], value=int(a)*0.000061, inplace= True)

In [96]:
data.Price.replace(to_replace=['£ 310.40','£ 537.75','£ 439.00'], value=[274.7,459.6,375.2], inplace= True)

In [97]:
e

['About BTC 0.15/ETH 4.78(crypto curr)', 'C$889.00 / Rp3,450,000']

In [98]:
data.Price.replace(to_replace= 'About BTC 0.15/ETH 4.78(crypto curr)', value= 3151, inplace= True)
data.Price.replace(to_replace='C$ 889.00 / Rp 3,450,000', value= 889/1.06, inplace= True)

## Chipset
Một số sản phẩm có 2 chip cho 2 phiên bản khác nhau. Tuy nhiên sản phẩm hiện tại chỉ là phiên bản đầu tiên, vì vậy cần loại bỏ giá trị cho phiên bản khác

Các giá trị chứa tên của nhiều hãng hoặc tên hãng xuất hiện 2 lần là các sản phẩm cần tìm để làm sạch

In [99]:
# chip = {'Qualcom':['snapdragon'],
#        'MediaTek':['helio','dimensity'],
#        'Samsung':['exynos'],
#        'Huawei':['kirin','hisilicon'],
#        'Intel':['atom'],
#        'Apple':['bionic','fusion'],
#        'Nvidia':['tegra'],
#        'TI':['omap'],
#        'Unisoc':['spreadtrum','tiger']}

In [100]:
idf = data.index
lst_chip_b = ['qualcom','mediatek','samsung','huawei','intel','apple','nvidia','unisoc']
lst_chip = np.array(['hummingbird','snapdragon','helio','dimensity','exynos','kirin','hisilicon','atom','bionic','fusion','tegra','spreadtrum','omap','tiger'])

In [101]:
i_contain_char = []
i_contain_2chip = []
i_contain_2brand = []
for i in idf:
    s = data.at[i,'Chipset'].lower()
    check = ['or',',','.','/']
    fl = False
    for c in check:
        if c in s:
            i_contain_char.append(i)
            break
    if fl:
        continue
    su = 0
    for c in lst_chip:
        if c in s: 
            su+=1
        if su > 1: 
            break
    if su > 1: 
        i_contain_2chip.append(i)
        continue
    su = 0
    for c in lst_chip_b:
        if c in s: 
            su+=1
        if su > 1: 
            break
    if su > 1: 
        i_contain_2brand.append(i)
len(i_contain_char), len(i_contain_2chip), len(i_contain_2brand)

(138, 63, 14)

In [102]:
df_4 = data.copy()

In [103]:
for i in i_contain_2chip:
    chip = df_4.at[i,'Chipset'].split(')')[0]+')'
    if i == 2070: chip = chip[len('Exynos 1330 - SM-A146B'):]
    if i == 2868: chip = chip[len('Exynos 7578 '):]
    if i == 2883 or i == 2884: chip = chip[len('Exynos 5433 - T810, T815 '):]
    if i == 1172: chip = 'Qualcomm MSM8960 ' + chip[:13] + ' Pro'
    if i == 2870: chip = chip[len('Spreadtrum SC9830 '):]
#     print(i, chip, sep =': ')
    df_4.at[i,'Chipset'] = chip

In [104]:
for i in i_contain_char:
    chip = df_4.at[i,'Chipset'].split(')')[0]+')'
    chip = chip.split(' - ')[0]
    if i == 0: chip = chip[:16] + chip[18:]
    if i == 4730: chip = chip[:14]+chip[22:]
    if i == 4860: chip = chip[:18]+chip[29:]
    if i == 2962: chip = chip[:9]
    if i == 1198: chip = chip[:16]
    if i == 3215: chip = chip[:16] + chip[27:]
    if i==474: chip='Mediatek MT8125 (28 nm)'
    idx = chip.find('/')
    if idx != -1:
        chip = chip[:idx] + chip[idx+3:]
    i1,i2 = chip.find('('), chip.find(')')
    if i1 == -1 and i2 != -1: chip= chip[:-1]
#     print(i, chip, sep =': ')
    df_4.at[i,'Chipset'] = chip

In [105]:
for i in i_contain_2brand:
    s = df_4.at[i,'Chipset'].split(')')[0]+')'
    if i == 567 or i == 569: s='Mediatek MT6750 (28 nm)'
    if i == 587: s='Mediatek MT6737M (28 nm)'
    if i ==595: s='Mediatek MT6580M (28 nm)'
    if i==596: s=s.replace('- K120E, K130EQualcomm MSM8909 Snapdragon 210 ','')
    if i==1746:s=s.replace('- 3G modelQualcomm MSM8916 Snapdragon 410 ','')
    if i==2519:s=s.replace('Qualcomm Snapdragon 410 ','')
    if i ==4807:s=s.replace(' Qualcomm MSM8939 Snapdragon 615','')
#     print(i, s, sep =': ')
    df_4.at[i,'Chipset'] = s

In [106]:
i_dup_brand=[]
for i in idf:
    s = df_4.at[i,'Chipset'].lower()
    for c in lst_chip_b:
        if s.count(c) > 1:
            i_dup_brand.append(i)
            break
len(i_dup_brand)

39

In [107]:
for i in i_dup_brand:
    s = df_4.at[i,'Chipset'].split(')')[0]+')'
    s = s.split(' - ')[0]
    if s.find('(') == -1 and 'Mediatek' in s: s+=' (28 nm)'
    if i==4566:s='Qualcomm MSM8956 Snapdragon 650 (28 nm)'
    if i==286 or i==300: s='Intel Atom Z3580'
    if i==1059: s=s.replace('Mediatek Helio P15','')
    if i==3434:s=s.replace('Mediatek MT6580M ','')
    if i==3537:s=s.replace(' Mediatek Dimensity 920','')
#     print(i,s,sep=': ')
    df_4.at[i,'Chipset']=s

## OS

Loại bỏ các thông tin sau dấu phảy

In [108]:
df_5=df_4.copy()

In [109]:
i_dup_os=[]
i_ct_char=[]
for i in idf:
    os = df_5.at[i,'OS']
    s=os.split(',')[0]
    df_5.at[i,'OS'] = s
    nameOs=os.split(' ')[0]
    if s.count(nameOs) > 1:
        i_dup_os.append(i)
        continue
    for ch in ['-','/',';','\\','or','x']:
        if ch in s:
            i_ct_char.append(i)
            break
len(i_dup_os), len(i_ct_char)

(13, 15)

In [110]:
for i in i_dup_os:
    os = df_5.at[i,'OS']
    os = os.split('/')[0].split(' - ')[0].split(' or')[0]
    if ')' in os:
        os=os.split(')')[0]+')'
#     print(i,os,sep=': ')
    df_5.at[i,'OS'] = os

In [111]:
for i in i_ct_char:
    os = df_5.at[i,'OS'].split(' - ')[0]
    if i==1078: os='Android 5.0 (Lollipop)'
    if i==3267 or i==3269: os='Android 12'
    if i==2474: os='Android 7.0 (Nougat)'
    if i==2620:os='Microsoft Windows Mobile 5.0'
    if i==1137: os='Android 2.0'
    if i==3472: os ='Android 5.0.2 (Lollipop)'
    if i==3757: os= 'Android 6.0 (Marshmallow)'
    if i==3530: os='Android 2.5'
    os=os.split('/')[0]
    df_5.at[i,'OS'] = os

## Video MC

In [112]:
df_5.Status[data['Video MC'] =='Yes'].value_counts()

Discontinued    331
Available        18
Name: Status, dtype: int64

Các giá trị này đa phần đều ngừng sản xuất. Bởi có nhiều dữ liệu và khó để bổ sung từ nhiều nguồn. Vì vậy chọn phương pháp xoá

In [113]:
df_5.drop(df_5.Status[data['Video MC'] =='Yes'].index,inplace=True)

In [114]:
idf= df_5.index

Tách chất lượng video và khung hình video. Lấy dữ liệu về chất lượng video tốt nhất

In [115]:
for i in idf:
    s = df_5.at[i,'Video MC']
    lst = s.split(', ')
    best_res = lst[0].split('@')[0]
    max_fps = -1
    for q in lst:
        lst_tmp = q.split('@')
        if len(lst_tmp) > 1:
            ss = lst_tmp[1]
            if 'fps' in ss:
                ss = ss[:-3]
            max_fps = max(max_fps, np.max(np.array(re.findall("\d+", ss)).astype(int)))
    df_5.at[i,'Video MC'] = best_res

In [116]:
df_5.dtypes

ID              int64
Name           object
Brand          object
Announced      object
Status         object
Size          float64
Resolution     object
OS             object
Chipset        object
Card slot      object
ROM            object
RAM            object
Num of MC       int64
Pic MC         object
Video MC       object
Num of SC       int64
Pic SC         object
3.5mm jack     object
Battery        object
Price         float64
ScreenArea    float64
ppi           float64
dtype: object

In [117]:
df_5.drop(columns=['ID']).to_csv('dataClearRaw.csv',index=False)

## Ram, Rom
Từ file code 'crawlRAMROM', ta thu thập lại dữ liệu thô chính xác về Ram,Rom. Dữ liệu này được lưu dưới dạng csv.
Thứ tự trong dữ liệu này trùng với thứ tự của dữ liệu hiện tại

In [118]:
df = pd.read_csv('dataClearRaw.csv')
dfrrc = pd.read_csv('ramromchip4533.csv')

In [119]:
rom = []
ram = []
for i in range(len(dfrrc)):
    l=dfrrc.at[i,'RAM']
    romraw = l.split(',')[0]
    ro = romraw.split('/')[0].split(' ')[0]
    ramraw = dfrrc.at[i,'ROM']
    ra = ramraw.split('/')[0].split('-')[0].split(' ')[0]
    if ra.isnumeric():
        if 'GB' in ramraw: ra += 'GB'
        elif 'MB' in ramraw: ra += 'MB'
    if ra == '\xa0': ra = '-1'
    rom.append(ro)
    ram.append(ra)
#     print(i,romraw, ra, sep=' -:- ')

Để dễ dàng xử lý và xác nhận với nguồn thu thập, nên thêm link của sản phẩm

In [120]:
lst_link = []
with open('linkdata.txt') as f:
    lst_link = f.readlines()
llink = []
for v in lst_link:
    llink.append(v.replace('\n',''))
len(llink)

4533

In [121]:
df.RAM = ram
df.ROM = rom
df['Link'] = llink

Xoá các bản ghi có RAM = -1 do thiếu thông tin trên web và đa phần đều là sản phẩm cũ, không còn thông tin 

In [122]:
df.drop(df[df.RAM =='-1'].index,inplace=True)
df.reset_index(inplace=True,drop=True)

In [123]:
df.shape

(4505, 22)

## Pic MC/SC
Lấy giá trị cao nhất tương ứng với chất lượng chụp ảnh tốt nhất của sản phẩm

In [124]:
for i in df.index:
    s = df.at[i,'Pic SC']
    l = np.array(s.split(',')).astype(float)
    if len(l)!=1: df.at[i,'Pic SC'] = max(l)
    s = df.at[i,'Pic MC']
    l = np.array(s.split(',')).astype(float)
    if len(l)!=1: df.at[i,'Pic MC'] = max(l)

In [125]:
df['Pic SC'] = df['Pic SC'].astype(float)
df['Pic MC'] = df['Pic MC'].astype(float)

In [131]:
df.dtypes

Name           object
Brand          object
Announced      object
Status         object
Size          float64
Resolution     object
OS             object
Chipset        object
Card slot      object
ROM            object
RAM            object
Num of MC       int64
Pic MC        float64
Video MC       object
Num of SC       int64
Pic SC        float64
3.5mm jack     object
Battery       float64
Price         float64
ScreenArea    float64
ppi           float64
Link           object
dtype: object

# Lưu dữ liệu đã làm sạch

In [132]:
df.to_csv('CleanData.csv',index=False)

In [4]:
df = pd.read_csv('CleanData.csv')
df[df.columns[:12]].head()

,Name,Brand,Announced,Status,Size,Resolution,OS,Chipset,Card slot,ROM,RAM,Num of MC
0,1T 7,alcatel,2018-02-01,Available,7.0,600 x 1024,Android 8.1 (Oreo),Mediatek MT8321A (28 nm),microSDXC,8GB,1GB,1
1,Idol 5s,alcatel,2017-09-01,Available,5.2,1080 x 1920,Android 7.1 (Nougat),Mediatek MT6757 Helio P20 (16 nm),microSDXC,32GB,3GB,1
2,Idol 5,alcatel,2017-09-01,Available,5.2,1080 x 1920,Android 7.1 (Nougat),Mediatek MT6753 (28 nm),microSDXC,16GB,3GB,1
3,A7 XL,alcatel,2017-09-01,Available,6.0,1080 x 1920,Android 7.1 (Nougat),Mediatek MT6750S (28 nm),microSDHC,32GB,3GB,2
4,A7,alcatel,2017-09-01,Available,5.5,1080 x 1920,Android 7.0 (Nougat),Mediatek MT6750T (28 nm),microSDXC,32GB,3GB,1


In [5]:
df[df.columns[12:]].head()

,Pic MC,Video MC,Num of SC,Pic SC,3.5mm jack,Battery,Price,ScreenArea,ppi,Link
0,2.0,720p,1,0.3,Yes,2580.0,70.0,137.9,170.0,https://www.gsmarena.com/alcatel_1t_7-9094.php
1,12.0,1080p,1,8.0,Yes,2850.0,140.0,74.5,424.0,https://www.gsmarena.com/alcatel_idol_5s-8844.php
2,13.0,1080p,1,5.0,Yes,2800.0,120.0,74.9,423.0,https://www.gsmarena.com/alcatel_idol_5-8832.php
3,12.0,1080p,1,5.0,Yes,4000.0,260.0,99.2,367.0,https://www.gsmarena.com/alcatel_a7_xl-8843.php
4,16.0,1080p,1,8.0,Yes,4000.0,190.0,83.4,401.0,https://www.gsmarena.com/alcatel_a7-8830.php
